In [109]:
import os 
import pandas as pd
import numpy as np
import re, html
import requests
# from bs4 import BeautifulSoup as BS, NavigableString, SoupStrainer
from bs4 import BeautifulSoup as bs 
from html_table_parser import parser_functions
from pororo import Pororo
import time
import pickle
from tqdm import tqdm

In [107]:
EV = pd.read_csv('/home/kic/data/kodex/EV_processed.csv', dtype=object)

sungwook = EV.loc[0:23, 'corp_code']
sangwoo = EV.loc[23:45, 'corp_code']
ahreum = EV.loc[45:, 'corp_code']

# 각자 이름 넣기!
name = EV.corp_code

### HTML crawling & parsing

In [110]:
# docs = {
#     corp_code: {
#         html: html,
#         sections: [list of sections],
#         ner: [list of ner results]
# }
pickle_file = '/home/kic/data/dart_processed/html_EV.pickle'

if os.path.isfile(pickle_file):
    with open(pickle_file, 'rb') as f:
        docs = pickle.load(f)
else:
    
    docs = dict() 

base_url = 'https://dart.fss.or.kr'
for code in tqdm(name):
    # stock_code = '00536541'
    # stock_code = '00490090'

    url = f'{base_url}/navi/searchNavi.do?naviCrpCik={code}&naviCode=A002'
    response = requests.get(url=url)

    soup = bs(response.text, "html.parser")

    new_url = f"{base_url}{soup.find('iframe')['src']}"
    new_response = requests.get(url=new_url)


    soup = bs(new_response.text, "html.parser")

    section_tags = soup.findAll('p', {'class': 'section-2'})
    sections = []

    contents = str(soup)
    for i in range (1, len(section_tags)):
        start = contents.rfind(str(section_tags[i-1]))
        end = contents.rfind(str(section_tags[i]))
        temp = bs(contents[start:end], 'html.parser').text
        temp = re.sub('\n', ' ', temp)
        temp = re.sub('\\xa0', ' ', temp)
        temp = re.sub('\s+', ' ', temp)
        sections.append(temp)

    docs[code] = {
        'html': str(new_response.text),
        'sections': sections,
        'ner': []
    }
    time.sleep(60)
    

 81%|██████████████████████████████████▋        | 54/67 [55:57<13:29, 62.30s/it]Bad pipe message: %s [b'UE{\xc6\\\xf4\xcct=\x87\xcfX\xb0\xbf\x90S\xf6\x87 J\xf8\xd8', b'> \xeb\xb1\xf9Y\xb6a\xa8F\x9e\x8d\xbcm9\xa2T]N\xad%\xc9J\xc8\xbe\xf0\x1e\x00\x08\x13']
Bad pipe message: %s [b'\x03', b'\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127', b'.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00']
Bad pipe message: %s [b'\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01']
Bad pipe message: %s [b'+K_I2lR\xcdR\xac\xa8` #\xc5\x1a~7 UB\xd3\x07\x99\xf1\xb5\xd24\xc6\x10\x19\xd8\x9b?\xf9\x0e\xff\xc4w\xd92\xba\x19B~:\r\xa1\x16R\xb3\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04']
Bad pipe message: %s [b'\x01\x02']
Bad pipe message: %s [b"\xd4k\xf7\x14\x84\x82\

In [111]:
with open(pickle_file, 'wb') as f:
    pickle.dump(docs, f, pickle.HIGHEST_PROTOCOL)

In [114]:
for code, val in docs.items():
    if len(val['html']) < 100:
        print(code, val['html'])

### Summarize & NER

In [ ]:
# html 파일 불러오기 
# pickle_file = '/home/kic/data/dart_processed/html_EV.pickle'
# with open(pickle_file, 'rb') as f:
#     docs = pickle.load(f)

In [115]:
summ = Pororo(task="summarization", model="extractive", lang="ko")
ner = Pororo(task="ner", lang="ko")

In [142]:
for code, corp in tqdm(docs.items()):
    # 초기화 
    if corp['ner']:
        corp['ner'] = []  
    idx = 0
        
    for section in corp['sections']:
        sample = summ(section)
        sample_list = []

        # token 길이 조절 
        while len(sample) > 512:
            sample_list.append(sample[512*idx:512*(idx+1)])
            sample = sample[512*(idx+1)]

        sample_list.append(sample[512*idx:])

        
        corp['ner'].append(
            set([x for s in sample_list for x in ner(s) if x[1] not in  ['O', 'QUANTITY', 'DATE']])  # 중복 제거 
        )


100%|███████████████████████████████████████████| 67/67 [01:27<00:00,  1.31s/it]


In [143]:
with open(pickle_file, 'wb') as f:
    pickle.dump(docs, f, pickle.HIGHEST_PROTOCOL)